In [1]:
# 最初にpyjagsの使い方を習得
# SageMath kernelはだめ、Python 2 Kernel
import pyjags
import numpy as np

In [2]:
np.random.seed(0)
np.set_printoptions(precision=1)

N = 500
a = 70
b = 4
sigma = 50

x = np.random.uniform(0, 100, size=N)
y = np.random.normal(a + x*b, sigma, size=N)

code = '''
model {
    for (i in 1:N) {
        y[i] ~ dnorm(alpha + beta * x[i], tau)
    }
    alpha ~ dunif(-1e3, 1e3)
    beta ~ dunif(-1e3, 1e3)
    tau <- 1 / sigma^2
    sigma ~ dgamma(1e-4, 1e-4)
}
'''

In [3]:
model = pyjags.Model(code, data=dict(x=x, y=y, N=N), chains=4)
samples = model.sample(5000, vars=['alpha', 'beta', 'sigma'])

adapting: iterations 876 of 4000, elapsed 0:00:01, remaining 0:00:03
adapting: iterations 1864 of 4000, elapsed 0:00:02, remaining 0:00:02
adapting: iterations 2856 of 4000, elapsed 0:00:03, remaining 0:00:01
adapting: iterations 3844 of 4000, elapsed 0:00:04, remaining 0:00:00
adapting: iterations 4000 of 4000, elapsed 0:00:04, remaining 0:00:00
sampling: iterations 952 of 20000, elapsed 0:00:01, remaining 0:00:19
sampling: iterations 1932 of 20000, elapsed 0:00:02, remaining 0:00:18
sampling: iterations 2916 of 20000, elapsed 0:00:03, remaining 0:00:17
sampling: iterations 3908 of 20000, elapsed 0:00:04, remaining 0:00:16
sampling: iterations 4900 of 20000, elapsed 0:00:05, remaining 0:00:15
sampling: iterations 5892 of 20000, elapsed 0:00:06, remaining 0:00:14
sampling: iterations 6884 of 20000, elapsed 0:00:07, remaining 0:00:13
sampling: iterations 7880 of 20000, elapsed 0:00:08, remaining 0:00:12
sampling: iterations 8876 of 20000, elapsed 0:00:09, remaining 0:00:11
sampling: ite

In [4]:
def summary(samples, varname, p=95):
    values = samples[varname]
    ci = np.percentile(values, [100-p, p])
    print('{:<6} mean = {:>5.1f}, {}% credible interval [{:>4.1f} {:>4.1f}]'.format(
      varname, np.mean(values), p, *ci))

for varname in ['alpha', 'beta', 'sigma']:
    summary(samples, varname)

alpha  mean =  64.3, 95% credible interval [57.1 71.4]
beta   mean =   4.0, 95% credible interval [ 3.9  4.1]
sigma  mean =  49.4, 95% credible interval [46.9 52.1]
